# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

# Importing Datasets

In [4]:
train=pd.read_csv("train_dataset.csv")

In [5]:
test=pd.read_csv("test_dataset.csv")

# Extracting values

In [6]:
train.shape

(534, 11)

In [11]:
train.columns

Index(['Unnamed: 0', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'FamilyCount',
       'Title', 'Has Cabin', 'IsAlone', 'Survived'],
      dtype='object')

In [16]:
train=train.set_index('Unnamed: 0')
train.shape

(534, 10)

In [17]:
X_train=train.iloc[:,0:9]
X_train.head()

,Pclass,Sex,Age,Fare,Embarked,FamilyCount,Title,Has Cabin,IsAlone
Unnamed: 0,,,,,,,,,
711,1,0,29.699118,26.5500,0,0,1,1,0
466,2,0,29.699118,0.0000,0,0,1,0,0
213,2,0,30.000000,13.0000,0,0,1,0,0
496,1,1,54.000000,78.2667,2,1,2,1,1
583,1,0,36.000000,40.1250,2,0,1,1,0


In [18]:
y_train=train.Survived

In [22]:
test=pd.read_csv("test_dataset.csv")

In [23]:
test=test.set_index('Unnamed: 0')

In [25]:
X_test=test.iloc[:,0:9]
y_test=test.Survived

# LogisticRegression Model

In [26]:
lr=LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
acc_log = round(lr.score(X_train, y_train) * 100, 2)
acc_log

c:\Users\Admin\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


83.71

In [27]:
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score

In [29]:
# Make predictions on the training and validation sets
y_train_pred = lr.predict(X_train)
y_val_pred = lr.predict(X_test)
# Print empirical risk on both sets
print('MSE on training set:')
print(mean_squared_error(y_train, y_train_pred))
print('MSE on validation set:')
print(mean_squared_error(y_test, y_val_pred))
print('')

# Print R squared on both sets
print('R squared on training set:')
print(r2_score(y_train, y_train_pred))
print('R squared on validation set:')
print(r2_score(y_test, y_val_pred))

MSE on training set:
0.16292134831460675
MSE on validation set:
0.20670391061452514

R squared on training set:
0.31365509905597677
R squared on validation set:
0.14409408115792177
